In [1]:
import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt

In [24]:
cities = [random.sample(range(100), 2) for x in range(15)];

In [25]:
cities

[[92, 13],
 [30, 80],
 [51, 60],
 [65, 64],
 [41, 21],
 [67, 37],
 [88, 97],
 [61, 28],
 [15, 17],
 [83, 33],
 [64, 39],
 [36, 45],
 [8, 95],
 [34, 66],
 [57, 36]]

In [26]:
tour = random.sample(range(15),15);

In [2]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

In [3]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0
    
    def routeDistance(self):
        if self.distance ==0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance
    
    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness

In [4]:
def createRoute(cityList):
    route = random.sample(cityList, len(cityList))
    return route

In [5]:
def initialPopulation(popSize, cityList):
    population = []

    for i in range(0, popSize):
        population.append(createRoute(cityList))
    return population

In [6]:
def rankRoutes(population):
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)

In [7]:
def selection(popRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
    
    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

In [8]:
def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

In [9]:
def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []
    
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])
        
    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child

In [10]:
def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])
    
    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool)-i-1])
        children.append(child)
    return children

In [11]:
def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))
            
            city1 = individual[swapped]
            city2 = individual[swapWith]
            
            individual[swapped] = city2
            individual[swapWith] = city1
    return individual

In [12]:
def mutatePopulation(population, mutationRate):
    mutatedPop = []
    
    for ind in range(0, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

In [13]:
def nextGeneration(currentGen, eliteSize, mutationRate):
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration

In [14]:
def geneticAlgorithm(population, popSize, eliteSize, mutationRate, generations):
    pop = initialPopulation(popSize, population)
    print("Initial distance: " + str(1 / rankRoutes(pop)[0][1]))
    
    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, mutationRate)
    
    print("Final distance: " + str(1 / rankRoutes(pop)[0][1]))
    bestRouteIndex = rankRoutes(pop)[0][0]
    bestRoute = pop[bestRouteIndex]
    return bestRoute

In [22]:
cityList = []

for i in range(0,25):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

In [23]:
cityList

[(138,11),
 (112,76),
 (181,188),
 (54,98),
 (86,141),
 (74,137),
 (146,24),
 (114,193),
 (32,185),
 (113,124),
 (185,128),
 (53,59),
 (58,50),
 (71,67),
 (42,186),
 (190,196),
 (98,166),
 (189,67),
 (89,197),
 (188,191),
 (63,51),
 (163,190),
 (29,21),
 (195,83),
 (143,3)]

In [19]:
cityList = [(11003.6111,42102.5),(11108.6111,42373.8889),(11133.3333,42885.8333),(11155.8333,42712.5),(11183.3333,42933.3333),(11297.5,42853.3333),(11310.2778,42929.4444),(11416.6667,42983.3333),(11423.8889,43000.2778),(11438.3333,42057.2222),(11461.1111,43252.7778),(11485.5556,43187.2222),(11503.0556,42855.2778),(11511.3889,42106.3889),(11522.2222,42841.9444),(11569.4444,43136.6667),(11583.3333,43150),(11595,43148.0556),(11600,43150),(11690.5556,42686.6667),(11715.8333,41836.1111),(11751.1111,42814.4444),(11770.2778,42651.9444),(11785.2778,42884.4444),(11822.7778,42673.6111),(11846.9444,42660.5556),(11963.0556,43290.5556),(11973.0556,43026.1111),(12058.3333,42195.5556),(12149.4444,42477.5),(12286.9444,43355.5556),(12300,42433.3333),(12355.8333,43156.3889),(12363.3333,43189.1667),(12372.7778,42711.3889),(12386.6667,43334.7222),(12421.6667,42895.5556),(12645,42973.3333)]

In [ ]:
cities = [random.sample(range(100), 2) for x in range(15)];
tour = random.sample(range(10000),38);
for temperature in numpy.logspace(0,5,num=100000)[::-1]:
    [i,j] = sorted(random.sample(range(10000),2))
    newTour =  tour[:i] + tour[j:j+1] +  tour[i+1:j] + tour[i:i+1] + tour[j+1:]
    if math.exp( ( sum([ math.sqrt(sum([(cities[tour[(k+1) % 15]][d] - cities[tour[k % 15]][d])**2 for d in [0,1] ])) for k in [j,j-1,i,i-1]]) - sum([math.sqrt(sum([(cities[newTour[(k+1) % 15]][d] - cities[newTour[k % 15]][d])**2 for d in [0,1] ])) for k in [j,j-1,i,i-1]])) / temperature) > random.random():
8       tour = copy.copy(newTour);
9  plt.plot([cities[tour[i % 15]][0] for i in range(16)], [cities[tour[i % 15]][1] for i in range(16)], 'xb-');
10 plt.show()

In [20]:
cityList

[(11003.6111, 42102.5),
 (11108.6111, 42373.8889),
 (11133.3333, 42885.8333),
 (11155.8333, 42712.5),
 (11183.3333, 42933.3333),
 (11297.5, 42853.3333),
 (11310.2778, 42929.4444),
 (11416.6667, 42983.3333),
 (11423.8889, 43000.2778),
 (11438.3333, 42057.2222),
 (11461.1111, 43252.7778),
 (11485.5556, 43187.2222),
 (11503.0556, 42855.2778),
 (11511.3889, 42106.3889),
 (11522.2222, 42841.9444),
 (11569.4444, 43136.6667),
 (11583.3333, 43150),
 (11595, 43148.0556),
 (11600, 43150),
 (11690.5556, 42686.6667),
 (11715.8333, 41836.1111),
 (11751.1111, 42814.4444),
 (11770.2778, 42651.9444),
 (11785.2778, 42884.4444),
 (11822.7778, 42673.6111),
 (11846.9444, 42660.5556),
 (11963.0556, 43290.5556),
 (11973.0556, 43026.1111),
 (12058.3333, 42195.5556),
 (12149.4444, 42477.5),
 (12286.9444, 43355.5556),
 (12300, 42433.3333),
 (12355.8333, 43156.3889),
 (12363.3333, 43189.1667),
 (12372.7778, 42711.3889),
 (12386.6667, 43334.7222),
 (12421.6667, 42895.5556),
 (12645, 42973.3333)]

In [21]:
geneticAlgorithm(population=cityList, popSize=38, eliteSize=20, mutationRate=0.01, generations=500)

AttributeError: 'tuple' object has no attribute 'distance'